### This is a simple notebook to build, visualize, and diagnose the performance of DT algorithms on the (larger) habitable planets data set.

It accompanies Chapter 3 of the book.

Data for this exercise come from [the Planet Habitability Lab](https://phl.upr.edu/projects/habitable-exoplanets-catalog).

Copyright: Viviana Acquaviva (2023)

Additions and Modifications by Julieta Gruszko (2025)

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

#### List the names your group members below:

In [ ]:
import pandas as pd
import numpy as np

import sklearn.tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

from scipy import stats

In [ ]:
from io import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
matplotlib.rcParams['figure.dpi'] = 300

pd.set_option('display.max_columns', 100) #These ensure that we can visualize all rows and columns in large data frames
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

### Step 1: Preliminary data analysis/exploration.

Once we are working with research-level data sets, our first step should always be data exploration.

We can read the data in a data frame, as we did previously, and do some preliminary data analysis.

In [ ]:
df = pd.read_csv('../Data/phl_exoplanet_catalog.csv', sep = ',')

In [ ]:
df.head()

We can visualize the names of available features like this:

In [ ]:
df.columns

The "describe" property is very useful to visualize some summary statistics.

In [ ]:
df.describe()

Here is a way of showing summary statistics by class.

In [ ]:
df.groupby('P_HABITABLE').count()

### Questions:
How many non-habitable planets are in our learning set? What about optimistically habitable planets (indicated by P_HABITABLE = 1) and reasonably expected to be habitable (P_HABITABLE = 2) planets?

Is the data set balanced or imbalanced?

### We start by lumping together Probably and Possibly Habitable planets, so we obtain a binary classification problem.

In [ ]:
bindf = df.drop('P_HABITABLE', axis = 1) #What are we doing here? Creating a new data frame called bindf and droppoing the old habitability tag

In [ ]:
bindf['P_HABITABLE'] = (np.logical_or((df.P_HABITABLE == 1) , (df.P_HABITABLE == 2))) #How about here? Creating the new habitability tag

bindf['P_HABITABLE'] = bindf['P_HABITABLE'].astype(int) #And here? Re-casting this column as integer

In [ ]:
bindf.head()

### There are several columns that we can use! Here is a sample.

S_MAG - star magnitude 

S_DISTANCE - star distance (parsecs)

S_METALLICITY - star metallicity (dex)

S_MASS - star mass (solar units)

S_RADIUS - star radius (solar units)

S_AGE - star age (Gy)

S_TEMPERATURE - star effective temperature (K)

S_LOG_G - star log(g)

P_DISTANCE - planet mean distance from the star (AU) 

P_FLUX - planet mean stellar flux (earth units)

P_PERIOD - planet period (days) 

### We will only select the three features we used in Chapter 2.

In [ ]:
final_features = bindf[['S_MASS', 'P_PERIOD', 'P_DISTANCE']] 

In [ ]:
targets = bindf.P_HABITABLE

In [ ]:
final_features.head()

### There are some missing values (NaNs); we can see this in different ways, one of them is by using the "describe" property, which only counts the numerical values in each column.

In [ ]:
final_features.shape

In [ ]:
final_features.describe()

### We can count missing data by column...

In [ ]:
final_features.isnull().sum() #can also use .isna

### ...and get rid of them (Note: there are much better imputing strategies!)

In [ ]:
final_features = final_features.dropna(axis = 0) #gets rid of any instance with at least one NaN in any column
final_features.shape

### Questions:
- Which of the 3 columns we chose as features would you expect to be most difficult to measure? (Use the missing values of the learning set to make an educated guess.)
- How many instances in our original learning set are missing at least one of the three selected features? In other words, how much data (and what fraction of our learning set) did we lose by choosing this imputing strategy?

### Next step: search for outliers.

Method 1 - plot!

In [ ]:
plt.hist(final_features.iloc[:,0], bins = 100, alpha = 0.5)

Non-pro tip: when you plot a distribution in a histogram, and the x-range seems unnecessarily large, it means that there are outliers! :) 

A good way to see them is to switch to a log scale on the y axis. Make that edit in the plotting code above.

In this case, there is a remarkable outlier, as you can see by looking at the distribution ouput from plt.hist; the same happens for other features. 

But we could have also known from the difference between mean and median (which, in fact, is even more pronounced for orbital distance and period).

In [ ]:
final_features.describe()

This eliminates objects with > 5 sigma outliers in any column; it counts the number of standard deviations away from the mean.

In [ ]:
final_features = final_features[(np.abs(stats.zscore(final_features)) < 5).all(axis=1)] 

### Question:
Why might this not be the ideal approach? Think about how outliers affect the distribution in question. What might be a better approach?

In [ ]:
targets = targets[final_features.index]

### Now we reset the index of the data frame

In [ ]:
final_features = final_features.reset_index(drop=True)

In [ ]:
final_features.head()

### and do the same for the label vector.

In [ ]:
targets = targets.reset_index(drop=True)

In [ ]:
targets.head()

### Comparing the shapes, we can see that 9 outliers were eliminated.

In [ ]:
targets.shape

### Check balance of data set

In [ ]:
#Simple way: count 0/1s, get fraction of total
np.sum(targets)/len(targets)

In [ ]:
np.bincount(targets) #this shows the distribution of the two classes

### This tells us that our data set is extremely imbalanced, and therefore, we need to be careful.

### Questions:
- What fraction of our learning set do the Habitable Planets make up?
- Imagine we create a "lazy classifier" that classifies all the instances as non-habitable. What would the accuracy of this classifier be?

#### Finally, we can explore the data by class, to get a sense of how the two classes differ from one another. For this, we need to concatenate the feature/labels data frames, so we group by objects label.

In [ ]:
#Note: this generates a "view", not a new data frame

pd.concat([final_features, targets], axis=1)

### We can group objects by label and take a look at summary statistics.


In [ ]:
pd.concat([final_features, targets], axis=1).groupby('P_HABITABLE').describe(percentiles = [])

#### We can also just take a look at the first two features, using different colors for the two classes.

In [ ]:
plt.figure(figsize=(10,6))

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#20B2AA','#FF00FF'])

a = plt.scatter(final_features['S_MASS'], final_features['P_PERIOD'], marker = 'o',\
            c = targets, s = 10, cmap=cmap, alpha = 0.5, label = 'Test')

plt.legend();


plt.yscale('log')
plt.xlabel('Mass of Parent Star (Solar Mass Units)')
plt.ylabel('Period of Orbit (days)');

bluepatch = mpatches.Patch(color='#20B2AA', label='Not Habitable')
magentapatch = mpatches.Patch(color='#FF00FF', label='Habitable')

ax = plt.gca()
leg = ax.get_legend()

plt.legend(handles=[magentapatch, bluepatch],\
           loc = 'lower right', fontsize = 14);

### Questions: 

- Do the two classes (habitable and not habitable planets) show any population-level difference in the 3 features we selected? In other words, do we have any hope of classifying the instances using these 3 features? If they completely match, no ML algorithm will be able to successfully classify instances with only this information. 

- Based on this graph, would you expect DT or kNN to perform better? Why?
    
- What kind of performance can we expect (qualitatively, is the information sufficient?) Do you expect to have latent (hidden) variables that might affect the outcome beyond those that we have?




### Ok, this is all for preliminary data exploration. Time to deploy.

We begin with a random train/test split, and will do cross validation later.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(final_features, targets, random_state=2)
Xtrain.shape, Xtest.shape


### Question: 
What fraction of our learning set is being used as test data? Where is this being set?

Let's pick the DT method (fixing random state) and build the model.

In [ ]:
model = DecisionTreeClassifier(random_state=5)

model.fit(Xtrain, ytrain)

#### Let's visualize the graph!

In [ ]:
# Reminder: The features are always randomly permuted at each split. 
# Therefore, the best found split may vary, even with the same training data 
# and max_features=n_features, if the improvement of the criterion is identical 
# for several splits enumerated during the search of the best split. 
# To obtain a deterministic behaviour during fitting, random_state has to be fixed.

dot_data = StringIO()
export_graphviz(
            model,
            out_file =  dot_data,
            feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'],
            class_names = ['Not Habitable','Habitable'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
nodes = graph.get_node_list()

for node in nodes:
    if node.get_label():
        values = [int(float(ii)) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]
        values = [255 * v / sum(values) for v in values]
        
        values = [int(255 * v / sum(values)) for v in values]
            
        if values[0] > values[1]:
            alpha = int(values[0] - values[1])
            alpha = '{:02x}'.format(alpha) #turn into hexadecimal
            color = '#20 B2 AA'+str(alpha)
        else:
            alpha = int(values[1] - values[0])
            alpha = '{:02x}'.format(alpha)
            color = '#FF 00 FF'+str(alpha)
        node.set_fillcolor(color)

#graph.write_png('Graph.png',dpi = 300)
        
Image(graph.create_png())

### Question: Can you predict the accuracy score on the train set?

### Let's take a look at train/test scores.

In [ ]:
print(metrics.accuracy_score(ytrain, model.predict(Xtrain))) #train score

print(metrics.accuracy_score(ytest, model.predict(Xtest))) #test score

This looks pretty high, but how does it compare with the accuracy of a lazy classifier that places everything in the "not habitable" category?

In [ ]:
#Dummy classifier would return all 0's as targets

print(metrics.accuracy_score(ytest, np.zeros(len(ytest)))) #performance of a dummy classifier


### We can look at other metrics.

In [ ]:
print(metrics.precision_score(ytest,model.predict(Xtest)))

In [ ]:
print(metrics.recall_score(ytest,model.predict(Xtest)))

How are they overall?

Not great.




### You know what we would need in order to understand exactly how the model is working? A confusion matrix!

In [ ]:
# This code just makes the plot. You have to pass it the confusion matrix and list of classes.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(7,6))
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="green" if i == j else "red", fontsize = 30)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### We can plot the confusion matrix.

Note that so far, we use the predictions on *one* test fold, so the numbers will refer to the test set only.

In [ ]:
cm = metrics.confusion_matrix(ytest,model.predict(Xtest))

plot_confusion_matrix(cm, ['Not Hab','Hab'], cmap = plt.cm.Pastel2)

### We can now implement three flavors of k-fold Cross Validation.

Note: you can fix the random seed for exactly reproducible behavior.

The three methods below are:
- Standard version, no shuffling or stratification
- Shuffled version, without stratification
- Shuffled and stratified version, ensures that class distributions resemble the entire data set

You should avoid the first one of these! Sometimes our data sets will have instances ordered by their labels, so not shuffling can be dangerous. 

In [ ]:
# This is the standard version. Important: it doesn't shuffle the data, 
# so if your positive examples are all at the beginning or all the end, it might lead to disastrous results.

cv1 = KFold(n_splits = 5)

#This is v2: shuffling added (recommended!)

cv2 = KFold(shuffle = True, n_splits = 5, random_state=5)

# STRATIFICATION ensures that the class distributions in each split resembles those of the 
# entire data set 

cv3 = StratifiedKFold(shuffle = True, n_splits = 5, random_state=5)


### Effect of stratification: let's look at the class count in each set of splits.

In [ ]:
for train, test in cv1.split(final_features, targets): #Just how they are in original data set
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv2.split(final_features, targets): #One random selection
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv3.split(final_features, targets): #One adjusted-for random selection
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

### The handy function cross\_validate provides the scores (specified by the chosen scoring parameter), in dictionary form.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'accuracy')

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'accuracy')

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'accuracy')

In [ ]:
scores1

#### We can now calculate an average and standard deviation.

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['test_score'].std()))

In [ ]:
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['test_score'].std()))

In [ ]:
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['test_score'].std()))

#### Question: are the differences statistically significant?

### Let's now use recall as our scoring parameter. Will the model that is the outcome of the training step change?

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv1, scoring = 'recall')

scores2 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv2, scoring = 'recall')

scores3 = cross_validate(DecisionTreeClassifier(random_state=1), final_features, targets, cv = cv3, scoring = 'recall')

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['test_score'].std()))
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['test_score'].std()))
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['test_score'].std()))

### If desired, we can ask for the train scores as well. This is very helpful when diagnosing bias vs variance.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'recall', \
                         return_train_score = True)

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'recall', \
                         return_train_score = True)

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'recall', 
                         return_train_score = True)

In [ ]:
print("{:.3f}".format(scores1['test_score'].mean()), "{:.3f}".format(scores1['train_score'].mean()))
print("{:.3f}".format(scores2['test_score'].mean()), "{:.3f}".format(scores2['train_score'].mean()))
print("{:.3f}".format(scores3['test_score'].mean()), "{:.3f}".format(scores3['train_score'].mean()))

Unsurprisingly, the recall scores on the train set are perfect. Note that while we generally can't predict one metric from the other (for example, we can't generally predict what recall is if we know the accuracy), when the accuracy score is 100%, it means that the model doesn't make any mistakes, so precision and recall will also be 100%.

### The cross\_validate function is useful to calculate the score, but does not produce predicted labels.

#### These can be obtained by using the cross\_val\_predict function, which saves the predictions for each of the k test folds, and compiles them together.

In [ ]:
model1 = DecisionTreeClassifier(random_state = 3)

y1 = cross_val_predict(model1, final_features, targets, cv = cv1) #these are the predictions,
                                                                #and they are independent of the scoring parameter!

This output is useful to build the "full" confusion matrix:

In [ ]:
metrics.confusion_matrix(targets,y1)

### However, things may change if I use a different cross validation scheme:

In [ ]:
model1 = DecisionTreeClassifier(random_state = 3)

y1 = cross_val_predict(model1, final_features, targets, cv = cv1)

In [ ]:
model2 = DecisionTreeClassifier(random_state = 3)

y2 = cross_val_predict(model2, final_features, targets, cv = cv2)

In [ ]:
np.sum(y1-y2)

In [ ]:
np.sum(y1)

In [ ]:
metrics.confusion_matrix(targets,y1)

In [ ]:
metrics.confusion_matrix(targets,y2)

This is a good reminder that the confusion matrix is also only one possible realization of the model, and is subject to random fluctuations just like the cross validation scores.

### Finally, we can plot learning curves, using this handy function from sklearn.

Learning curves are helpful in order to visualize the training scores vs the test scores, and how they vary as a function of data set size. They allow us to determine whether we have enough learning data, AND whether we have a high bias or high variance problem.

The source code below is a slight modification of [this code](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html).

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=5,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), scoring = 'accuracy'):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure(figsize=(10,6))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel(str(scoring))
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score from cross-validation")

    plt.legend(loc="best")
    return plt

In [ ]:
model = DecisionTreeClassifier(random_state = 5)

In [ ]:
plot_learning_curve(model, 'DT (default params)', final_features, targets,  cv = cv3, scoring = 'recall');

### Questions:
- How many times was the model trained to create the plot above? In other words, how many calls to $ \texttt{fit} $ did the  $\texttt{plot\_learning\_curve} $ function make? This is an important number to know ahead of time if you're using a computationally intensive model!

- What is being shown by the filled bands in the plot above?

### Conclusions: 
- How is our DT model doing? Does it suffer from high variance or high bias? 
- What would you suggest we do to improve the model? Make at least 2 suggestions.

#### We won't look at the kNN classifier here, but we'll follow up with it on Homework 2!

Chapter 3 of the book discusses additional applications, like the kNN algorithm results, and the case of a 3-class classifier.

### Acknowledgement Statement:

### Upload your completed notebook to Gradescope to submit it, and then you're done for today.